In [7]:
import pandas as pd
import networkx as nx
from pyvis.network import Network

In [11]:
meta = pd.read_csv("series-networks/meta.csv")
edgelist = pd.read_csv("series-networks/gameofthrones/edges.csv")
attr = pd.read_csv("series-networks/gameofthrones/attributes.csv")

In [3]:
major_house_df = attr[attr["house"].notna()]

In [12]:
major_houses = ["Targaryen", "Stark", "Lannister",
                "Baratheon", "Greyjoy", "Tully",
                "Arryn", "Martell", "Tyrell"]

In [13]:
def find_major_house(houses):
    for house in houses:
        house_names = ''.join(c if c.isalnum() else ' ' for c in house).split()
        for name in house_names:
            if name in major_houses:
                return name
    return None

In [14]:
major_house_df["major_house"] = major_house_df["house"].str.split().apply(find_major_house)
major_house_df = major_house_df[major_house_df["major_house"].notna()]

id_set = set(major_house_df["id"])
major_houses_edgelist = edgelist[edgelist["from"].isin(id_set) &
                                 edgelist["to"].isin(id_set)]
major_houses_edgelist = major_houses_edgelist[major_houses_edgelist["from"] != major_houses_edgelist["to"]]
g_major_houses = nx.from_pandas_edgelist(major_houses_edgelist, "from", "to")

id_house_dict = dict(zip(major_house_df["id"], major_house_df["major_house"]))
nx.set_node_attributes(g_major_houses, id_house_dict, "major_house")

In [15]:
degrees = dict(g_major_houses.degree())
top_30_nodes = sorted(degrees, key=degrees.get, reverse=True)[:30]
g_top_30 = g_major_houses.subgraph(top_30_nodes)

In [16]:
net = Network(notebook=True)
net.repulsion() # Stop the nodes from jumping around

net.from_nx(g_top_30)

net.toggle_physics(True)
net.show("got_characters_houses.html")

got_characters_houses.html
